In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

# Setup

In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# third party
from apis import make_submit_query
from apis import make_test_query
from helpers import create_user
from helpers import make_user

# syft absolute
import syft as sy
from syft import test_settings

In [ ]:
SERVER_PORT = "8080"
SERVER_URL = f"http://localhost:{SERVER_PORT}"

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port=SERVER_PORT,
    n_consumers=4,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url=SERVER_URL, email="info@openmined.org", password="changethis"
)

In [ ]:
num_users = 2
users = []
email_disable_index = 0
for i in range(num_users):
    user = make_user()
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    users.append(user)

In [ ]:
mock_func = make_test_query(
    settings={
        "rate_limiter_enabled": True,
        "calls_per_min": 10,
    }
)

private_func = make_test_query(
    settings={
        "rate_limiter_enabled": False,
    }
)

new_endpoint = sy.TwinAPIEndpoint(
    path="bigquery.test_query",
    description="This endpoint allows to query Bigquery storage via SQL queries.",
    private_function=private_func,
    mock_function=mock_func,
    worker_pool="default-pool",
)

high_client.custom_api.add(endpoint=new_endpoint)

In [ ]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_1_col_id = test_settings.get("table_1_col_id", default="table_id")
table_1_col_score = test_settings.get("table_1_col_score", default="colname")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")

In [ ]:
submit_query_function = make_submit_query(
    settings={},
    worker_pool="default-pool",
)

high_client.custom_api.add(endpoint=submit_query_function)

# Create jobs

In [ ]:
# third party
from job_helpers import TestJob
from job_helpers import create_jobs
from job_helpers import submit_job

In [ ]:
n_per_user = 3

jobs = create_jobs(users, n_per_user=n_per_user)

print(f"num jobs: {len(jobs)}")
for job in jobs[:2]:
    print(f"Job type: {job.job_type}, should succeed: {job.should_succeed}")

assert len(jobs) == len(users) * n_per_user
assert all(isinstance(j, TestJob) for j in jobs)
assert all(job.client is not None for job in jobs)

In [ ]:
responses = []

for job in jobs:
    responses.append(submit_job(job))

In [ ]:
assert all(job.is_submitted for job in jobs)

## Test: cannot execute

In [ ]:
# Blocking

for job in jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result = execute_code_fn()

In [ ]:
# Nonblocking

for job in jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    result_job = execute_code_fn(blocking=False)
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result_job.wait()